# Exercice 1 -- correction
Numérotez les vers du sonnet en exemple à l'aide d'un attribut `@n` ajouté à chaque vers. Sauvegardez le résultat sous `exercice_1.xml`.

Nous allons donc devoir récupérer chaque vers, et trouver un moyen de calculer le nombre de vers antérieurs. Nous utiliserons deux méthodes différentes pour cela.


## 1 - Imports et *parsing* du document

In [1]:
import lxml.etree as etree
import copy

In [2]:
fichier = 'fichier_xml_1.xml'
document_as_xml = etree.parse(fichier)
print(document_as_xml)

## 2 - Récupération de la liste de vers
On va effectuer une copie de notre racine pour montrer deux manière d'arriver à notre résultat.

In [3]:
racine = document_as_xml.getroot()
racine_copiee = copy.deepcopy(racine)

In [4]:
all_verses = racine.xpath("//l")
print(all_verses)

[]


Ici, ça ne marche pas: on n'a pas spécifié les espaces de noms des éléments que l'on cherche. Commençons par attribuer un préfixe à notre espace de nom TEI:

In [5]:
tei_uri = "http://www.tei-c.org/ns/1.0"
namespaces_dict = {'tei': tei_uri}

Puis on peut relancer notre commande en modifiant la requête XPath pour y inclure les préfixes de la TEI:

In [6]:
all_verses = racine.xpath("//tei:l", namespaces=namespaces_dict)
print(all_verses)

[<Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239f00>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f72402391c0>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239180>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239140>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239100>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239080>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239040>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240239000>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240238fc0>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f72402390c0>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f7240238cc0>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f724023b640>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f724023b600>, <Element {http://www.tei-c.org/ns/1.0}l at 0x7f724023b540>]


Ça marche ! Il nous faut maintenant trouver une manière de compter les lignes. 

### Option A: avec XPath pur
La première option demande d'utiliser la fonction XPath 1.0 `count()` que l'on va appliquer à chacun de nos vers. Il suffira ensuite de manipuler le résultat pour le convertir en entier, puis en chaîne de caractère, pour enfin créer un attribut `@n` avec la valeur trouvée.

In [7]:
for verse in all_verses:
    # On ajoute 1 car on regarde les noeuds qui précèdent
    number = int(verse.xpath("count(preceding::tei:l)", namespaces=namespaces_dict)) + 1
    as_string = str(number)
    verse.set('n', as_string)
    print(as_string)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [8]:
sonnet = racine.xpath("//tei:lg[@type='poem']", namespaces=namespaces_dict)[0]

print(etree.tostring(sonnet, pretty_print=True).decode())

<lg xmlns="http://www.tei-c.org/ns/1.0" type="poem" met="-+ | -+ | -+ | -+ | -+ /">
               <head>
                  <title>Sonnet 17</title>
               </head>
               <lg type="sonnet" rhyme="abab cdcd efef gg">
                  <lg type="quatrain">
                     <l n="1">
                        <seg type="foot" real="+-"> Who will</seg>
                        <seg type="foot"> believe</seg>
                        <seg type="foot"> my verse</seg>
                        <seg type="foot"> in time</seg>
                        <seg type="foot"> to come,</seg>
                     </l>
                     <l n="2">
                        <seg type="foot">If it</seg>
                        <seg type="foot"> were fill&#8203;'d</seg>
                        <seg type="foot"> with your</seg>
                        <seg type="foot"> most high</seg>
                        <seg type="foot" real="+-"> deserts?</seg>
                     </l>
                   

Cela fonctionne parfaitement, mais suppose de lancer une expression XPath sur chacune des lignes. Or, l'exécution du XPath est relativement lente, et cela se verra si vous avez des milliers d'items à parser. Pouvons nous trouver une solution plus simple qui n'utilise que python?

### Option B - avec un traitement de liste simple

Notre liste contient en effet l'ensemble des vers qui nous intéressent. On peut donc l'utiliser pour attribuer à chacun des vers son numéro correspondant. Nous travaillons ici sur la copie du document originel, pour être certain de ne pas être parasités par l'option A.

In [9]:
all_verses_copies = racine_copiee.xpath("//tei:l", namespaces=namespaces_dict)

In [10]:
for index, verse in enumerate(all_verses_copies):
    # Attention à l'index qui commence à 0 en python !
    correct_value = index + 1
    verse.set('n', str(correct_value))

In [11]:
sonnet = racine_copiee.xpath("//tei:lg[@type='poem']", namespaces=namespaces_dict)[0]

print(etree.tostring(sonnet, pretty_print=True).decode())

<lg xmlns="http://www.tei-c.org/ns/1.0" type="poem" met="-+ | -+ | -+ | -+ | -+ /">
               <head>
                  <title>Sonnet 17</title>
               </head>
               <lg type="sonnet" rhyme="abab cdcd efef gg">
                  <lg type="quatrain">
                     <l n="1">
                        <seg type="foot" real="+-"> Who will</seg>
                        <seg type="foot"> believe</seg>
                        <seg type="foot"> my verse</seg>
                        <seg type="foot"> in time</seg>
                        <seg type="foot"> to come,</seg>
                     </l>
                     <l n="2">
                        <seg type="foot">If it</seg>
                        <seg type="foot"> were fill&#8203;'d</seg>
                        <seg type="foot"> with your</seg>
                        <seg type="foot"> most high</seg>
                        <seg type="foot" real="+-"> deserts?</seg>
                     </l>
                   

### Conclusion
Il y avait deux possibilités valides pour cet exercice. Du point de vue des performances, pour un texte de cette taille, la différence est minime. Pour des corpus plus massifs, je recommande cependant d'**optimiser le recours aux requêtes XPath** en les produisant le plus haut et le moins de fois possible, et de traiter ensuite les résultats à l'aide des manipulations de listes, de sets ou de dictionnaires que permet python.